In [154]:
from collections import Counter
words = open("names.txt", "r").read().splitlines()

counter = Counter()
for word in words:
  chs = "." + word + "."
  for c1, c2, c3 in zip(chs, chs[1:], chs[2:]):
    trigram = (c1, c2, c3)
    counter[trigram] += 1
print(counter.most_common(5))

[(('a', 'h', '.'), 1714), (('n', 'a', '.'), 1673), (('a', 'n', '.'), 1509), (('o', 'n', '.'), 1503), (('.', 'm', 'a'), 1453)]


In [155]:
import torch.nn.functional as F
xs, ys = [], []

for word in words:
  chs = list("." + word + ".")
  for c1, c2, c3 in zip(chs, chs[1:], chs[2:]):
    idx1 = stoi[c1]
    idx2 = stoi[c2]
    idx3 = stoi[c3]
    xs.append((idx1, idx2))
    ys.append(idx3)

# tensor function returns the same type as its original
xs = torch.tensor(xs)
ys = torch.tensor(ys)

xenc = F.one_hot(xs, num_classes=27).float()

In [156]:
xs.shape

torch.Size([196113, 2])

In [157]:
ys.shape

torch.Size([196113])

In [161]:
xenc.shape

torch.Size([196113, 2, 27])

In [162]:
W.shape


torch.Size([27, 27, 27])

In [165]:
g = torch.Generator().manual_seed(420)
W = torch.randn((27, 27, 27), generator=g, requires_grad=True)
# log-counts
logits = W[xs[:, 0], xs[:, 1], ...]
# counts
counts = logits.exp()
# probability
probs = counts / counts.sum(1, keepdim=True)
probs

tensor([[0.1219, 0.0087, 0.0157,  ..., 0.0374, 0.1633, 0.0162],
        [0.0143, 0.0108, 0.0063,  ..., 0.0264, 0.0188, 0.0181],
        [0.0192, 0.0517, 0.0178,  ..., 0.0071, 0.0315, 0.0358],
        ...,
        [0.0030, 0.0031, 0.0078,  ..., 0.0260, 0.0391, 0.0318],
        [0.0515, 0.0135, 0.0502,  ..., 0.0560, 0.0064, 0.0056],
        [0.0192, 0.0194, 0.0069,  ..., 0.0718, 0.0414, 0.0128]],
       grad_fn=<DivBackward0>)

In [167]:
lr = 10
steps = 200

for i in range(steps):
    # forward pass
    logits = W[xs[:, 0], xs[:, 1], ...]
    counts = logits.exp()
    probs = (counts / counts.sum(1, keepdim=True))
    probs = probs[torch.arange(len(ys)), ys]
    loss = -probs.log().mean() + 0.01 * (W**2).mean()

    # backward pass
    W.grad = None
    loss.backward()
    W.data += -lr * W.grad

    if i % 10 == 0:
        print(f'{i} loss={loss.item():.4f}')
print(f'{i} loss={loss.item():.4f}')

0 loss=2.6713
10 loss=2.6553
20 loss=2.6403
30 loss=2.6261
40 loss=2.6128
50 loss=2.6001
60 loss=2.5881
70 loss=2.5766
80 loss=2.5657
90 loss=2.5554
100 loss=2.5454
110 loss=2.5360
120 loss=2.5269
130 loss=2.5182
140 loss=2.5099
150 loss=2.5019
160 loss=2.4942
170 loss=2.4868
180 loss=2.4797
190 loss=2.4728
199 loss=2.4669


In [170]:
xs, ys = [], []

for w in words:
    chs = ['.', *list(w), '.']
    for c1, c2, c3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[c1]
        ix2 = stoi[c2]
        ix3 = stoi[c3]
        xs.append((ix1, ix2))
        ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
n = len(ys)
xs.shape, ys.shape

(torch.Size([196113, 2]), torch.Size([196113]))

In [172]:
idxs = torch.randperm(n)
idxs_tra = idxs[:int(n*0.8)]
idxs_val = idxs[int(n*0.8):int(n*0.9)]
idxs_tst = idxs[int(n*0.9):]
idxs_tra.shape, idxs_val.shape, idxs_tst

(torch.Size([156890]),
 torch.Size([19611]),
 tensor([190443,  65949,  15673,  ..., 101980,   9866,  74836]))

In [174]:
W = torch.randn((27 * 2, 27), requires_grad=True)
xenc_train = F.one_hot(xs[idxs_tra, :], num_classes=27).float().view(-1, 27*2)
xenc_val = F.one_hot(xs[idxs_val, :], num_classes=27).float().view(-1, 27*2)
xenc_test = F.one_hot(xs[idxs_tst, :], num_classes=27).float().view(-1, 27*2)
y_train = ys[idxs_tra]
y_val = ys[idxs_val]
y_test = ys[idxs_tst]
W.shape, xenc_train.shape, xenc_val.shape, xenc_test.shape

(torch.Size([54, 27]),
 torch.Size([156890, 54]),
 torch.Size([19611, 54]),
 torch.Size([19612, 54]))

In [175]:
lr = 10
steps = 200

for i in range(steps):
    # forward pass
    logits = xenc_train @ W
    counts = logits.exp()
    prob = (counts / counts.sum(1, keepdim=True))
    prob = prob[torch.arange(len(y_train)), y_train]
    loss = -prob.log().mean() + 0.01 * (W**2).mean()
    

    with torch.no_grad():
        vlogits = xenc_val @ W
        vcounts = vlogits.exp()
        vprob = (vcounts / vcounts.sum(1, keepdim=True))
        vprob = vprob[torch.arange(len(y_val)), y_val]
        val_loss = -vprob.log().mean()
    
    # backward pass
    W.grad = None
    loss.backward()
    W.data += -lr * W.grad
    

        

    if i % 10 == 0:
        print(f'{i} train_loss={loss.item():.4f} | val_loss={val_loss.item():.4f}')
print(f'{i} train_loss={loss.item():.4f} | val_loss={val_loss.item():.4f}')

0 train_loss=4.2478 | val_loss=4.2451
10 train_loss=3.0335 | val_loss=3.0364
20 train_loss=2.7437 | val_loss=2.7439
30 train_loss=2.6129 | val_loss=2.6109
40 train_loss=2.5374 | val_loss=2.5338
50 train_loss=2.4878 | val_loss=2.4830
60 train_loss=2.4526 | val_loss=2.4470
70 train_loss=2.4262 | val_loss=2.4200
80 train_loss=2.4057 | val_loss=2.3990
90 train_loss=2.3894 | val_loss=2.3823
100 train_loss=2.3760 | val_loss=2.3686
110 train_loss=2.3649 | val_loss=2.3572
120 train_loss=2.3555 | val_loss=2.3476
130 train_loss=2.3474 | val_loss=2.3394
140 train_loss=2.3405 | val_loss=2.3322
150 train_loss=2.3344 | val_loss=2.3260
160 train_loss=2.3290 | val_loss=2.3205
170 train_loss=2.3242 | val_loss=2.3156
180 train_loss=2.3199 | val_loss=2.3113
190 train_loss=2.3161 | val_loss=2.3073
199 train_loss=2.3130 | val_loss=2.3041


In [176]:
g = torch.Generator().manual_seed(420)
for _ in range(5):
    name = ''
    ix1 = stoi['.']
    ix2 = stoi['.']
    while True:
        
        xen = F.one_hot(torch.tensor((ix1, ix2)), 27).float().view(-1, 27*2)
        logits = xen @ W
        counts = logits.exp()
        prob = (counts / counts.sum(1, keepdims=True))
        ix3 = torch.multinomial(prob, num_samples=1, generator=g).item()
        if ix3 == stoi['.']:
            break
        name += itos[ix3]
        ix1 = ix2
        ix2 = ix3
    print(name)

raiusal
leerstari
iuamad
ya
amonzatlixann
